In [1]:
#Import Packages
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objs as go

#Import Team Data and helper functions
from constants import TEAM_TO_TEAM_ABBR, TEAM_SETS, CURRENT_TEAM

In [2]:
decades = [1990 + i*10 for i in range(4)] 
filename_list = ['../data/raw/NBA_' + str(decades[i]) + '-'+ str(decades[i+1]) + '_Game-outcomes.csv' for i in range(len(decades) - 1)]

li = []
for filename in filename_list:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

dec = pd.concat(li, axis=0, ignore_index=True)
dec['Date'] = pd.to_datetime(dec['Date'], format="%Y-%m-%d")

In [3]:
#Initialise the elo
elo = {team[0] : [] for team in CURRENT_TEAM}
starting_elo = 1000

for team in CURRENT_TEAM:
  start = min(dec["Date"].loc[dec["Winner"] == team[0]].min(),dec["Date"].loc[dec["Loser"] == team[0]].min()) - timedelta(days=1)
  elo[team[0]].append([start, starting_elo])

dec = dec.dropna()

#Replace old names
replace_key = {}
for i in range(len(TEAM_SETS)):
  for old_name in TEAM_SETS[i]:
    replace_key[old_name] = CURRENT_TEAM[i]

dec = dec.replace({col:replace_key for col in ["Winner", "Loser", "HomeTeam"]})

In [4]:
#Define ELO Functions

def expected_outcome(elo_a,elo_b,m):
  #returns E_A
  return 1/(1 + 10**((elo_b - elo_a)/m))

def update_elo(elo_a, elo_b, s_a, k, m):
  return (elo_a + k*(s_a - expected_outcome(elo_a,elo_b,m)))


def elo_formula(game, elo,k = 10,m = 400):
  winner = game['Winner']
  loser = game['Loser']
  date = game['Date']

  winner_elo = elo[winner][-1][1]
  loser_elo = elo[loser][-1][1]
  
  winner_elo_new = update_elo(winner_elo, loser_elo, 1, k, m)
  loser_elo_new = update_elo(loser_elo, winner_elo, 0, k, m)

  elo[winner].append([date, winner_elo_new])
  elo[loser].append([date, loser_elo_new])

  return elo


In [5]:

for i in range(len(dec)):
  elo = elo_formula(dec.iloc[i,:], elo)

df = pd.DataFrame()

for team in CURRENT_TEAM:
  df_team = pd.DataFrame(elo[team[0]])
  df_team.columns = ['Date', 'Elo']
  df_team['Team'] = team[0]
  df = pd.concat([df,df_team])

In [6]:
#Generate Plotting Information

first  = df['Date'].min()
last = df['Date'].max()
current = df['Date'].min()
job_done = False
day_remove = []


while not job_done:
  if current not in df['Date']:
    day_remove.append(current)
  
  if current == last:
    job_done = True
  
  current +=  timedelta(days=1)

#Other
dt_all = pd.date_range(start=first,end=last)
dt_obs = [d.strftime("%Y-%m-%d") for d in df['Date']]
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

In [10]:
fig = go.Figure()
for team in CURRENT_TEAM:
  fig.add_trace(go.Scatter(x=df['Date'].loc[df['Team'] == team[0]],
                           y=df['Elo'].loc[df['Team'] == team[0]],
                    mode='lines',
                    # hoverinfo='skip',
                    name=team[0]))

fig.update_xaxes(
    # type = 'category',
    # rangeslider_visible=True,
    rangebreaks = [dict(values=dt_breaks)],
    # range = dt_obs,
    # rangeselector=dict(
    #     buttons=list([
    #         dict(count=1, label="1m", step="month", stepmode="backward"),
    #         dict(count=6, label="6m", step="month", stepmode="backward"),
    #         dict(count=1, label="YTD", step="year", stepmode="todate"),
    #         dict(count=1, label="1y", step="year", stepmode="backward"),
    #         dict(step="all")
    #     ])
    # )
)


fig.write_html("../report/figures/NBA_ELO_Standard_1990-2020.html")